In [31]:
import pandas as pd
import numpy as np #для матричных вычислений

from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn import ensemble #ансамбли

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

In [32]:
data=pd.read_csv('D:\Irina\Data Science\ML\Оптимизация гиперпараметров модели\data\_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


Создаем матрицу наблюдений $X$ и вектор ответов $y$.

In [33]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

Разделяем выборку на тренировочную и тестовую в соотношении 80/20. Для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение). 

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)
print('X_train:{}'.format(X_train.shape))
print('y_train:{}'.format(y_train.shape))
print('X_test:{}'.format(X_test.shape))
print('y_test:{}'.format(y_test.shape))

X_train:(3000, 1776)
y_train:(3000,)
X_test:(751, 1776)
y_test:(751,)


Для подбора гиперпараметров разделим тестовую выборку на валидационную и тестовую.

In [35]:
X_valid, X_test_2, y_valid, y_test_2 = train_test_split(X_test, y_test, stratify=y_test, random_state = 42, test_size = 0.5)
print('X_valid:{}'.format(X_valid.shape))
print('X_test_2:{}'.format(X_test_2.shape))

X_valid:(375, 1776)
X_test_2:(376, 1776)


Обучим модель на всех данных при помощи логистической регрессии

In [36]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter=50)

#Обучаем модель
log_reg.fit(X_train, y_train)
y_train_pred=log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)
#Выводим значения метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.88
f1_score на тестовом наборе: 0.78


c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Проделаем аналогичное для RandomForestClassifier().

In [37]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 1.00
f1_score на тестовом наборе: 0.80


На полном наборе данных метрика выше для RandomForestClassifier().

Подберем гиперпараметры методом GridSeachCV для логистической регрессии

In [38]:
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_1.fit(X_valid, y_valid) 

y_test_2_pred = grid_search_1.predict(X_test_2)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_2, y_test_2_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

CPU times: total: 1.5 s
Wall time: 16.9 s
f1_score на тестовом наборе: 0.74
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Обучение на параметрах, подобранных методом GridSearchCV ухудшило метрику f1

Подберем гиперпараметры методом GridSeachCV для случайного леса.

In [39]:
param_grid = {'n_estimators': list(range(50, 200, 10)),
              'min_samples_leaf': [3, 5, 7],
              'max_depth': list(np.linspace(10, 50, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_forest.fit(X_valid, y_valid) 
y_test_2_pred = grid_search_forest.predict(X_test_2)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_2, y_test_2_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: total: 5.89 s
Wall time: 1min 9s
f1_score на тестовом наборе: 0.77
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 3, 'n_estimators': 110}


Для случайного леса значение метрики так же ухудшилось

Подберем гиперпараметры методом RandomizedSearchCV для логистической регрессии

In [40]:
param_distributions = {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]},
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search.fit(X_valid, y_valid) 
y_test_2_pred = random_search.predict(X_test_2)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_2, y_test_2_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 109 ms
Wall time: 2.54 s
f1_score на тестовом наборе: 0.69
Наилучшие значения гиперпараметров: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}


Подбор параметров методом RandomizedSearchCV для логистической регрессии на данный момент показал самые низкие результаты.

Подберем гиперпараметры методом RandomizedSearchCV для случайного леса

In [41]:
param_distributions = {'n_estimators': list(range(50, 200, 10)),
                       'min_samples_leaf': [3, 5, 7],
                       'max_depth': list(np.linspace(10, 50, 5, dtype=int))
                       }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_valid, y_valid) 
y_test_2_pred = random_search_forest.predict(X_test_2)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_2, y_test_2_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 391 ms
Wall time: 4.43 s
f1_score на тестовом наборе: 0.76
Наилучшие значения гиперпараметров: {'n_estimators': 150, 'min_samples_leaf': 5, 'max_depth': 20}


Для случайного леса получили метрику чуть лучше, чем при методе GridSeachCV, но значительно быстрее.

Подберем гиперпараметры методом Hyperopt для логистической регрессии

In [42]:
random_state = 42
def hyperopt_lr(params, cv=5, X=X_valid, y=y_valid, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': str(params['penalty']), 
              'solver': str(params['solver']), 
              'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    #score = metrics.f1_score(y,model.predict(X))
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [65]:
%%time

# зададим пространство поиска гиперпараметров
pen = ['l1', 'l2']
sol = ['liblinear', 'saga']
space={'penalty': hp.choice('penalty', ['l1', 'l2']), # тип регуляризации
       'solver': hp.choice('solver', ['liblinear', 'saga']), # алгоритм оптимизации
       'C': hp.uniform('C', low=0.01, high=1) # уровень силы регурялизации
      }

CPU times: total: 0 ns
Wall time: 0 ns


In [68]:
%%time

trials = Trials() # используется для логирования результатов

# зададим пространство поиска гиперпараметров
pen = ['l1', 'l2']
sol = ['liblinear', 'saga']
space_lr={'penalty': hp.choice('penalty', ['l1', 'l2']), # тип регуляризации
       'solver': hp.choice('solver', ['liblinear', 'saga']), # алгоритм оптимизации
       'C': hp.uniform('C', low=0.01, high=1) # уровень силы регурялизации
      }
best=fmin(hyperopt_lr, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

# model = linear_model.LogisticRegression(
#     random_state = random_state, 
#     penalty = str(best['penalty']),
#     solver = str(best['solver']),
#     C = float(best['C'])
# )
# model.fit(X_valid, y_valid)
# y_valid_pred = model.predict(X_train_scaled)
# print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
# y_test_pred = model.predict(X_test)
# print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

 10%|█         | 2/20 [00:01<00:10,  1.78trial/s, best loss: -0.9778869778869779]

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 20%|██        | 4/20 [00:02<00:08,  1.99trial/s, best loss: -0.9778869778869779]

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 30%|███       | 6/20 [00:02<00:05,  2.34trial/s, best loss: -0.9778869778869779]

...

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 90%|█████████ | 18/20 [00:08<00:01,  1.86trial/s, best loss: -0.9902439024390245]

c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|██████████| 20/20 [00:09<00:00,  2.14trial/s, best loss: -0.9902439024390245]
Наилучшие значения гиперпараметров {'C': 0.9878598398404349, 'penalty': 1, 'solver': 0}
CPU times: total: 7.92 s
Wall time: 9.37 s


c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



In [72]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    random_state=random_state, 
    penalty=pen[best['penalty']],
    solver=sol[best['solver']],
    C=float(best['C'])
)
model.fit(X_valid, y_valid)
y_valid_pred = model.predict(X_valid)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.99
accuracy на тестовом наборе: 0.83
f1_score на тестовом наборе: 0.84


С помощью метода Hyperopt на данный момент получили самое высокое значение метрики

Подберем гиперпараметры методом Hyperopt для случайного леса

In [79]:
# зададим пространство поиска гиперпараметров
space_rf={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_valid, y=y_valid, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
              'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [81]:
%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [00:07<00:00,  2.73trial/s, best loss: -1.0]               
Наилучшие значения гиперпараметров {'max_depth': 18.0, 'min_samples_leaf': 2.0, 'n_estimators': 103.0}
CPU times: total: 6.06 s
Wall time: 7.34 s


In [82]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_valid, y_valid)
y_valid_pred = model.predict(X_valid)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 1.00
accuracy на тестовом наборе: 0.87
f1_score на тестовом наборе: 0.88


Для случайного леса hyperopt также показал наилучший результат

Подберем гиперпараметры методом Optuna для логистической регрессии

In [83]:
import optuna

print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.1.0


In [88]:

%%time

def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l1', 'l2']) 
  solver = trial.suggest_categorical('solver', ['liblinear', 'saga']) 
  C = trial.suggest_float('C',low=0.01, high=1,step=0.01,log=False)
  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                            solver=solver,
                            C=C,
                            random_state=random_state)
  # обучаем модель
  model.fit(X_valid, y_valid)
  score = metrics.f1_score(y_valid, model.predict(X_valid))

  return score


# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=20)

print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state, )
model.fit(X_valid, y_valid)
y_test_pred = model.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

[I 2023-03-25 19:00:44,933] A new study created in memory with name: LogisticRegression
[I 2023-03-25 19:00:45,036] Trial 0 finished with value: 0.9607843137254901 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.93}. Best is trial 0 with value: 0.9607843137254901.
c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2023-03-25 19:00:45,784] Trial 1 finished with value: 0.9680589680589682 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.19}. Best is trial 1 with value: 0.9680589680589682.
[I 2023-03-25 19:00:45,840] Trial 2 finished with value: 0.926470588235294 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.61}. Best is trial 1 with value: 0.9680589680589682.
[I 2023-03-25 19:00:45,899] Trial 3 finished with value: 0.9533169533169533 and parameters: {'penalty': 'l1', 'solver': 'liblinear',

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'saga', 'C': 0.6}
f1_score на обучающем наборе: 0.98
accuracy на тестовом наборе: 0.82
f1_score на тестовом наборе: 0.84
CPU times: total: 9.8 s
Wall time: 10.9 s


c:\Users\oikuz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


accuracy чуть ниже, чем при применении hyperopt, f1 на уровне

Подберем гиперпараметры методом Optuna для случайного леса

In [86]:

%%time
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 15, 26, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_valid, y_valid)
  score = metrics.f1_score(y_valid, model.predict(X_valid))

  return score


# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_valid, y_valid)
y_test_pred = model.predict(X_test)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

[I 2023-03-22 23:13:07,457] A new study created in memory with name: RandomForestClassifier
[I 2023-03-22 23:13:07,865] Trial 0 finished with value: 0.9227053140096618 and parameters: {'n_estimators': 143, 'max_depth': 21, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.9227053140096618.
[I 2023-03-22 23:13:08,551] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 197, 'max_depth': 22, 'min_samples_leaf': 2}. Best is trial 1 with value: 1.0.
[I 2023-03-22 23:13:08,915] Trial 2 finished with value: 0.9343065693430657 and parameters: {'n_estimators': 134, 'max_depth': 23, 'min_samples_leaf': 7}. Best is trial 1 with value: 1.0.
[I 2023-03-22 23:13:09,320] Trial 3 finished with value: 0.943765281173594 and parameters: {'n_estimators': 149, 'max_depth': 19, 'min_samples_leaf': 6}. Best is trial 1 with value: 1.0.
[I 2023-03-22 23:13:09,717] Trial 4 finished with value: 0.9208633093525179 and parameters: {'n_estimators': 156, 'max_depth': 23, 'min_samples_leaf': 9}. Be

Наилучшие значения гиперпараметров {'n_estimators': 197, 'max_depth': 22, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 1.00
accuracy на тестовом наборе: 0.87
f1_score на тестовом наборе: 0.88
CPU times: total: 7.73 s
Wall time: 9.54 s


метрики на уровне hyperopt

Таким образом, самые высоки метрики получились при применении методов Hyperopt и Optuna